In [ ]:
# --- Imports et fonctions ---
import os, sys, csv, json, time, urllib.parse, urllib.request
import pandas as pd
from google.colab import files

# Configuration
HTTP_TIMEOUT   = 30.0
GEO_THROTTLE   = 0.1
OSRM_THROTTLE  = 0.2
GEOCODE_CACHE  = {}
_last = 0.0

def throttle(sec):
    global _last
    dt = time.time() - _last
    if dt < sec: time.sleep(sec - dt)
    _last = time.time()

def fetch_json(url, timeout=HTTP_TIMEOUT):
    try:
        with urllib.request.urlopen(url, timeout=timeout) as r:
            return json.load(r)
    except Exception as e:
        print(f"[warn] requête échouée ({e}) : {url}")
        return None

def geocode_gouv(address):
    if address in GEOCODE_CACHE:
        return GEOCODE_CACHE[address]
    throttle(GEO_THROTTLE)
    q = urllib.parse.urlencode({"q":address,"limit":1})
    url = f"https://api-adresse.data.gouv.fr/search/?{q}"
    data = fetch_json(url)
    if data and data.get("features"):
        lon, lat = data["features"][0]["geometry"]["coordinates"]
        GEOCODE_CACHE[address] = (lat,lon)
        return lat, lon
    GEOCODE_CACHE[address] = (None,None)
    return None, None

def route_distance_osrm(a_lat,a_lon,b_lat,b_lon):
    throttle(OSRM_THROTTLE)
    coords = f"{a_lon},{a_lat};{b_lon},{b_lat}"
    url = f"https://router.project-osrm.org/route/v1/driving/{coords}?overview=false"
    data = fetch_json(url)
    if data and data.get("routes"):
        return round(data["routes"][0]["distance"]/1000,2)
    return None


In [ ]:
# --- Upload CSV ---
uploaded = files.upload()
csv_file = next(iter(uploaded.keys()))
print("Fichier reçu :", csv_file)


Saving fichier.csv to fichier (4).csv
Fichier reçu : fichier (4).csv


In [ ]:
import csv
import pandas as pd
from openpyxl import Workbook

# --- Traitement CSV ---
rows = list(csv.DictReader(open(csv_file, encoding="utf-8-sig")))
headers = rows[0].keys()
enriched_rows = []

total = len(rows)
for i, row in enumerate(rows, 1):
    addr1, addr2 = row["Adresse départ"], row["Adresse destination"]
    lat1, lon1 = geocode_gouv(addr1)
    lat2, lon2 = geocode_gouv(addr2)
    row["Lat départ"], row["Lon départ"] = lat1 or "", lon1 or ""
    row["Lat destination"], row["Lon destination"] = lat2 or "", lon2 or ""
    if lat1 and lat2:
        row["Distance_km"] = route_distance_osrm(lat1, lon1, lat2, lon2) or ""
    else:
        row["Distance_km"] = ""
    enriched_rows.append(row)
    print(f"Processed {i}/{total} → Adresse départ={row.get('Adresse départ','?')}")

# --- Écriture XLSX ---
output_file = csv_file.replace(".csv", "_with_coords.xlsx")
df = pd.DataFrame(enriched_rows)
df.to_excel(output_file, index=False)

# --- Téléchargement (si environnement Colab par exemple) ---
print("✅ Fichier enrichi :", output_file)
print(df.head())

try:
    from google.colab import files
    files.download(output_file)
except ImportError:
    print("📂 Fichier enregistré localement :", output_file)


Processed 1/143 → Adresse départ=122 RUE ROGER SALENGRO 59590 RAISMES
Processed 2/143 → Adresse départ=7 RUELLE DE LA MAIRIE 59860 BRUAY SUR ESCAUT
Processed 3/143 → Adresse départ=RESIDENCE NICOLAS DUBOIS APPARTEMENT 12 59230 SAINT AMAND LES EAUX
Processed 4/143 → Adresse départ=25 RESIDENCE CHATEAU D EAU 59230 SAINT AMAND LES EAUX
Processed 5/143 → Adresse départ=14 RUE DELATTRE DE TASSIGNY 59163 CONDE SUR L ESCAUT
Processed 6/143 → Adresse départ=27 RUE LEON BLUM 59230 SAINT AMAND LES EAUX
Processed 7/143 → Adresse départ=357 RUE DE L'ABBAYE 59590 RAISMES


KeyboardInterrupt: 